# Elastisches Fadenpendel

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook wird das Modell des mathematischen Pendels dadurch verallgemeinert, dass der Faden, an dem die schwingende Punktmasse befestigt ist, elastisch ist. Konkret wird der Faden durch eine gedämpfte Hooke'sche Feder mit einer nicht verschwindenden Ruhelänge modelliert. In diesem System werden zwei Schwingungen gekoppelt, wobei die Pendelschwingung nichtlinear ist, so dass eine analytische Lösung nicht möglich ist.

Die zu lösenden Bewegungsgleichungen lauten

\begin{aligned}
\ddot r &= r\dot\phi^2 + \cos(\phi) - D(r-1)-\kappa \dot{r}\\
\ddot\phi &= \frac{1}{r}\left[2\dot r\dot\phi + \sin(\phi)\right]\,.
\end{aligned}

## Importanweisungen

In [ ]:
from math import cos, pi, sin
import numpy as np
from scipy import integrate
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

## Implementierung des Differentialgleichungssystems

Die in der folgenden Funktion auftretenden Ausdrücke ergeben sich durch Umschreibung der beiden Bewegungsgleichungen in vier Differentialgleichungen 1. Ordnung.

In [ ]:
def dx_dt(t, x, d, kappa):
    r, phi, v_r, v_phi = x

    dr_dt = v_r
    dphi_dt = v_phi
    dvr_dt = r*v_phi**2 + cos(phi) - d*(r-1) - kappa*v_r
    dvphi_dt = -(2*v_r*v_phi + sin(phi)) / r

    return dr_dt, dphi_dt, dvr_dt, dvphi_dt

## Lösung des Differentialgleichungssystems

Die Lösung des Differentialgleichungssystems erfolgt mit Hilfe der SciPy-Funktion `integrate.solve_ivp`. Das Vorgehen wurde bereits in anderen Jupyter-Notebooks erläutert.

In [ ]:
def solution(t_end, n_out, d, kappa, r_0, phi_0,
             v_r_0, v_phi_0):
    t_values = np.linspace(0, t_end, n_out)
    x_0 = [r_0, phi_0, v_r_0, v_phi_0]
    solution = integrate.solve_ivp(
        dx_dt, (0, t_end), x_0, t_eval=t_values,
        args=(d, kappa))
    (r_values, phi_values, v_r_values,
     v_phi_values) = solution.y[:]
    return t_values, r_values, phi_values

## Implementierung der Bedienelemente und graphische Darstellung der Ergebnisse

Mit den Schiebereglern lassen sich die folgenden Parameter einstellen:
- `t_end`: Länge des zu betrachtenden Zeitintervalls
- `n_out`: Anzahl der zu betrachtenden Zeitpunkte
- `d`: dimensionslose Federkonstante
- `kappa`: dimensionslose Federdämpfungskonstante
- `r_0`: anfänglicher Abstand der Punktmasse vom Aufhängepunkt
- `phi_0`: anfänglicher Winkel
- `v_r_0`: anfängliche Radialgeschwindigkeit
- `v_phi_0`: anfängliche Winkelgeschwindigkeit

Als Ergebnisse werden die Zeitabhängigkeit des Winkels $\varphi(t)$ und des Abstands vom Aufhängepunkt $r(t)$ dargestellt.

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=20, min=5, max=30, step=5,
                   description=r"$\tau_\text{end}$"),
               "n_out":
               widgets.IntSlider(
                   value=1000, min=100, max=1000,
                   step=100, description=r"$n_\text{out}$"),
               "d":
               widgets.FloatSlider(
                   value=100, min=10, max=500, step=10,
                   description="$D$"),
               "kappa":
               widgets.FloatSlider(
                   value=1, min=0, max=10, step=0.1,
                   description=r"$\kappa$"),
               "r_0":
               widgets.FloatSlider(
                   value=0.2, min=0.1, max=2, step=0.1,
                   description="$r(0)$"),
               "phi_0":
               widgets.FloatSlider(
                   value=pi/4, min=-2, max=2, step=0.1,
                   description=r"$\phi(0)$"),
               "v_r_0":
               widgets.FloatSlider(
                   value=0, min=-1, max=1, step=0.1,
                   description="$v_r(0)$"),
               "v_phi_0":
               widgets.FloatSlider(
                   value=0, min=-2.5, max=2.5, step=0.1,
                   description=r"$v_\phi(0)$")
               }

@interact(**widget_dict)
def plot_result(t_end, n_out, d, kappa, r_0, phi_0,
                v_r_0, v_phi_0):
    t_values, r_values, phi_values = solution(
        t_end, n_out, d, kappa, r_0, phi_0, v_r_0, v_phi_0)

    fig, (ax1, ax2) = plt.subplots(2, 1)
    ax1.plot(t_values, phi_values)
    ax1.set_xlabel("$t$")
    ax1.set_ylabel(r"$\phi$")

    ax2.plot(t_values, r_values)
    ax2.set_xlabel("$t$")
    ax2.set_ylabel("$r$")